In [ ]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import scipy as sp
from sklearn import preprocessing

import utility_functions as fn

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Initialization

In [ ]:
# reads from csv into a new dataframe
raw_weather = pd.read_csv('weather_raw2.csv',index_col=1)

# reindexes by appropriate datetime
raw_weather.index = pd.to_datetime(raw_weather.index,format='%Y-%m-%d %H:00:00 +0000 UTC')
raw_weather.index.names = ['']

# removes unused columns
raw_weather.drop(raw_weather.columns[np.array([0,1,2,3,4,9,10,13,15,16,17,19,20,21,23,24,25,26])],axis=1,inplace=True)

# renames columns
column_names = ['temp','tmin','tmax','pressure','humidity','wind','rain','snow','clouds']
raw_weather.columns = column_names

#reorders columns
raw_weather = raw_weather[['temp','tmin','tmax','pressure','humidity','wind','clouds','rain','snow']]

In [ ]:
# drops duplicate rows
raw_weather.drop_duplicates(inplace=True)

In [ ]:
# TEMPORARY bypass missing 2014-2015 data
# weather = raw_weather.truncate('2016-01-01 00:00:00','2019-02-10 00:00:00')

weather = raw_weather.copy(deep=False)

In [ ]:
# add an 'impute_ok' column; True if time gap is less than 6 hours
orig_len = len(weather.index)

weather.insert(0,'impute_ok',True)

weather['impute_ok'].iloc[1:orig_len]=(weather.index[1:orig_len]-weather.index[0:orig_len-1] <= '06:00:00')

In [ ]:
# reindexes to add missing rows, for a total length of 53545
correct_dt = pd.DatetimeIndex(start='2013-01-01 00:00:00',end='2019-02-10 00:00:00',freq='h')
weather = weather.reindex(index=correct_dt)

# fills gaps in data with True if less than 6 hours, False if more
weather['impute_ok'].fillna(method='bfill',inplace=True)

In [ ]:
fn.plot_all(weather,'2013-01-02 00:00:00','2019-02-09 00:00:00')
weather.isnull().sum()

# Error omission

In [ ]:
# FINISH removing outliers, column by column probably

# weather['temp'].where(weather < weather.median()+6*weather.std(),inplace=True)
# weather['temp'].where(weather > weather.median()-4*weather.std(),inplace=True)




# assigns NaN precip values to zero, this is an unconfirmed assumption
weather['rain'].replace(np.NaN,0,inplace=True)
weather['snow'].replace(np.NaN,0,inplace=True)

# removes negative clouds measurements
weather['clouds'].clip(lower=0,inplace=True)

fn.plot_all(weather,'2013-01-02 00:00:00','2019-02-09 00:00:00')
weather.isnull().sum()

In [ ]:
weather.describe()

In [ ]:
# imputes only values that have impute_ok = True
# in the future, try linear vs quadratic here
# weather.mask(weather['impute_ok']==True, weather.interpolate(method='polynomial', order=7, limit=6), inplace=True)

In [ ]:
# plot over missing chunk to check that the simple impute has yielded expected results
fn.plot_feature(weather,'temp','2016-07-15 00:00:00','2016-07-25 00:00:00')

In [ ]:
# fn.gap_check(weather)

In [ ]:
# imputes large segments, ignores period for now
# to be replaced by FFT imputation in the future
weather = weather.interpolate(method='linear')

# removes impute_ok column
weather.pop('impute_ok')

fn.plot_feature(weather,'temp','2016-07-17 00:00:00','2016-07-23 00:00:00')

In [ ]:
# this function adds columns with data from previous hours associated with the target hour
hours_before = np.arange(16,25)
weather_with_previous = fn.add_hours_before(weather,hours_before)
weather_with_previous.describe()

In [ ]:
# implements sklearn MinMaxScaler
x = weather_with_previous.values
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
x_scaled = scaler.fit_transform(x)

scaled = pd.DataFrame(x_scaled)
scaled.columns = weather_with_previous.columns
scaled.index = weather_with_previous.index

scaled.describe()

In [ ]:
# scaled.to_csv('weather_clean.csv')

# unused code below

In [ ]:
# # returns index labels where nan values appear for a certain column
# nan_index = weather['temp'].index[weather['temp'].apply(np.isnan)]
# nan_index

# correct and complete datetime index for the date range considered
# dt = pd.DatetimeIndex(start='2013-01-01 00:00:00',end='2019-02-10 00:00:00',freq='h')

# # renames (here, makes lowercase) column labels using a simple loop
# df.columns = [x.lower() for x in df.columns]

# # implements sklearn scaler
# from sklearn import preprocessing
# x = df.values #returns a numpy array
# scaler = preprocessing.MinMaxScaler()
# x_scaled = scaler.fit_transform(x)
# df = pandas.DataFrame(x_scaled)